In [ ]:
from multiprocessing.pool import Pool
import numpy as np
import requests
import librosa
import json
import ray


In [ ]:
wav_path = "/data/codes/sb-apa/wav/One of the women is filling plates with cupcakes.wav"
transcript = "One of the women is filling plates with cupcakes"

waveform, sr = librosa.load(wav_path, sr=16000)

params = {
    "audio": waveform.tolist(),
    "transcript": transcript
}


In [ ]:
inputs = [params.copy() for i in range(500)]

@ray.remote
def send_query(params):
    response = requests.post("http://14.162.145.55:8686/scoring", json=params)
    return response

outputs = ray.get([send_query.remote(params) for params in inputs])

In [ ]:
import json

json_path = [
    "/data/codes/sb-apa/data/scoring/apa-type-9/filtered-data.json",
    "/data/codes/sb-apa/data/scoring/apa-type-10/filtered-data.json",
    "/data/codes/sb-apa/data/scoring/apa-type-12/filtered-data.json"
]

data = {}
for path in json_path:
    _data = json.load(open(path, "r"))
    for key, value in _data.items():
        assert key not in data
        
    data.update(_data)
print(len(data))

In [ ]:
from sklearn.model_selection import train_test_split

keys = list(data.keys())

In [ ]:
train_keys, val_keys = train_test_split(keys, test_size=0.05, random_state=42)

train_set = {key:data[key] for key in train_keys}
val_set = {key:data[key] for key in val_keys}

In [ ]:
len(val_set)

In [ ]:
data_dir = "/data/codes/sb-apa/data/scoring"

with open(f'{data_dir}/train.json', 'w', encoding='utf-8') as f:
    json_obj = json.dumps(train_set, indent=4, ensure_ascii=False)
    f.write(json_obj)
    
with open(f'{data_dir}/val.json', 'w', encoding='utf-8') as f:
    json_obj = json.dumps(val_set, indent=4, ensure_ascii=False)
    f.write(json_obj)